In [1]:
import numpy as np
import pandas as pd

from src.reader import Reader
from src.downloader import Downloader
from src.localreader import LocalReader, SHRINK, CENTERING, getMean

In [2]:
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Activation, Flatten, Dense, BatchNormalization
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
def get_cnn():
    x = Input(shape=(256, 256, 3))
    h = Conv2D(64, (11, 11), strides=(2, 2))(x)
    h = Activation('relu')(h)
    h = MaxPooling2D(pool_size=(2, 2))(h)
    h = BatchNormalization()(h)

    h = Conv2D(64, (5, 5), strides=(1, 1))(h)
    h = Activation('relu')(h)
    h = MaxPooling2D(pool_size=(2, 2))(h)
    h = BatchNormalization()(h)

    h = Conv2D(64, (3, 3), strides=(1, 1))(h)
    h = Activation('relu')(h)

    h = Conv2D(64, (3, 3), strides=(1, 1))(h)
    h = Activation('relu')(h)

    h = Flatten()(h)
    h = Dense(1000)(h)
    h = Activation('relu')(h)

    h = Dense(256)(h)
    h = Activation('relu')(h)

    p = Dense(10, activation="linear")(h)

    model = Model(inputs=x, outputs=p)
    
    return model

In [5]:
model = get_cnn()
model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.00001), metrics=['mse'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 123, 123, 64)      23296     
_________________________________________________________________
activation_3 (Activation)    (None, 123, 123, 64)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 61, 61, 64)        0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 61, 61, 64)        256       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 57, 57, 64)        102464    
_________________________________________________________________
activation_4 (Activation)    (None, 57, 57, 64)        0         
__________

In [ ]:
reader = LocalReader(width=256, height=256, shaping=SHRINK)
for images, scores in reader.read(start=0, end=100):
    mean_scores = getMean(scores)
    print(images.shape)
    model.fit(images, mean_scores, steps_per_epoch=3)


(154, 256, 256, 3)
Epoch 1/1
3/3 [==============================] - 29s 10s/step - loss: 21.1020 - mean_squared_error: 21.1020

(132, 256, 256, 3)
Epoch 1/1
3/3 [==============================] - 23s 8s/step - loss: 7.1566 - mean_squared_error: 7.1566

(184, 256, 256, 3)
Epoch 1/1
3/3 [==============================] - 31s 10s/step - loss: 3.9158 - mean_squared_error: 3.9158

(153, 256, 256, 3)
Epoch 1/1
3/3 [==============================] - 26s 9s/step - loss: 4.6977 - mean_squared_error: 4.6977

(66, 256, 256, 3)
Epoch 1/1
3/3 [==============================] - 12s 4s/step - loss: 3.1515 - mean_squared_error: 3.1515

(243, 256, 256, 3)
Epoch 1/1
1/3 [=========>....................] - ETA: 31s - loss: 1.5175 - mean_squared_error: 1.5175

In [5]:
import os
def get_tar_names():
    return [name.replace(".tar.gz", "") for name in os.listdir("data/images/original")]

In [6]:
tar_names = get_tar_names()

In [7]:
import random
def image_loader(batch_size, reader, tar_names):

    L = len(tar_names)

    #this line is just to make the generator infinite, keras needs that    
    while True:

        batch_start = 0
        batch_end = batch_size

        while batch_start < L:
            random_tar = random.randint(1, len(tar_names))
            
            for images, scores in reader.read(start=random_tar, end=random_tar+1):
                print(images.shape)
                mean_scores = getMean(scores)
                
                random_image_ixs = [random.randint(0, len(images)-1) for i in range(0,batch_size)]
                
                random_images = images[random_image_ixs]
                random_image_scores = mean_scores[random_image_ixs]
                yield (random_images, random_image_scores) #a tuple with two numpy arrays with batch_size samples     

            batch_start += 1

In [7]:
reader = LocalReader(width=256, height=256, shaping=SHRINK)

In [8]:

    model.fit_generator((images,scores), validation_data = (images,scores), steps_per_epoch=3, epochs=3)


Epoch 1/3


TypeError: 'tuple' object is not an iterator

In [ ]:
model.fit_generator(reader.read(start=0, end=100), validation_data = reader.read(start=0, end=3, ), validation_steps=1, steps_per_epoch=3, epochs=3)

Epoch 1/3









1/3 [=========>....................] - ETA: 38s - loss: 1363.4987 - mean_squared_error: 1363.4987

2/3 [===================>..........] - ETA: 14s - loss: 1081.9754 - mean_squared_error: 1081.9754




3/3 [==============================] - 46s 15s/step - loss: 1041.8793 - mean_squared_error: 1041.8793 - val_loss: 1265.4120 - val_mean_squared_error: 1265.4120
Epoch 2/3

3/3 [==============================] - 35s 12s/step - loss: 774.5340 - mean_squared_error: 774.5340 - val_loss: 681.7311 - val_mean_squared_error: 681.7311
Epoch 3/3



In [14]:
res = model.predict(val_images)

In [22]:
for i in range(len(res)):
    print("pred:", res[i][0], "true:", val_scores[i], "ix:", i)

pred: 5.8442802 true: 4.2591093117408905 ix: 0
pred: 5.143937 true: 5.44488188976378 ix: 1
pred: 5.8160887 true: 5.603174603174603 ix: 2
pred: 6.065269 true: 4.120967741935484 ix: 3
pred: 6.302983 true: 5.945945945945946 ix: 4
pred: 5.33452 true: 4.855371900826446 ix: 5
pred: 5.1396165 true: 5.5436507936507935 ix: 6
pred: 5.038646 true: 5.104 ix: 7
pred: 5.5324626 true: 5.26953125 ix: 8
pred: 4.883533 true: 4.54863813229572 ix: 9
pred: 5.3768 true: 5.874509803921568 ix: 10
pred: 5.236091 true: 3.57085020242915 ix: 11
pred: 6.706098 true: 4.2217573221757325 ix: 12
pred: 5.8227534 true: 6.058091286307054 ix: 13
pred: 5.349895 true: 5.004098360655738 ix: 14
pred: 5.579125 true: 5.723735408560311 ix: 15
pred: 5.1008162 true: 6.012195121951219 ix: 16
pred: 5.5452743 true: 5.755813953488372 ix: 17
pred: 6.194382 true: 6.124 ix: 18
pred: 5.0618825 true: 5.309236947791165 ix: 19
pred: 6.1361623 true: 5.336065573770492 ix: 20
pred: 5.653971 true: 5.108 ix: 21
pred: 5.5065284 true: 5.97244094488

In [25]:
from PIL import Image

data = val_images[90]
img = Image.fromarray(data, 'RGB')
img.show()